<a href="https://colab.research.google.com/github/TomasBlanzaco/Dise-o-de-Gu-as-pMODB/blob/main/Dise%C3%B1o_de_primers_pModB_(GUI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---


#Desarrollado por Tomás Blanzaco

---



#Instalación de widgets
1) Apretar el símbolo de play que aparece abajo a la izquierda de y van a empezar a descargarse widgets necesarios para que el programa funcione (esto se descarga en el entorno de Google Colab, no en la computadora así que no hay riesgo de virus!).

Después de descargarlo, bajas a la sección "Correr Programa"

In [ ]:
# @title
%pip install ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.2 MB/s eta 0:00:00


#Correr programa
Una vez descargados los widgets ya se puede correr el programa.

Para eso, hace click en el botón de "Play" de abajo a la izquierda.

Después de que hayas hecho click en el botón, ya podes utilizar el menú interactivo que se ve a continuación.

# - IMPORTANTE: Abajo de todo están las instrucciones por si hay alguna duda!!

In [ ]:
# @title
import sys
from ipywidgets import VBox, HBox, Label, Text, IntText, Button, Textarea, Dropdown, Layout, Output
from IPython.display import display

# Creo los widgets
module_dropdown = Dropdown(
    options=[('pMOD_B CSY4', 'csy4'), ('pMOD_B tRNA', 'trna')],
    description='Módulo:',
    disabled=False,
    layout=Layout(width='auto', display='flex', flex_flow='row', justify_content='space-between'),
    style={'description_width': 'initial'}
)

num_guides_input = IntText(
    description='Número de guías (sin NGG):',
    disabled=False,
    layout=Layout(width='auto', display='flex', flex_flow='row', justify_content='space-between'),
    style={'description_width': 'initial'}
)

# Creo VBox
guide_sequence_inputs_box = VBox([])

enzyme_dropdown = Dropdown(
    options=[('Esp3I', 1), ('BsaI', 2)],
    description='Enzima de restricción:',
    disabled=False,
    layout=Layout(width='auto', display='flex', flex_flow='row', justify_content='space-between'),
    style={'description_width': 'initial'}
)

check_conditions_button = Button(
    description='Chequear condiciones',
    disabled=False,
    button_style='',
    layout=Layout(width='auto', display='flex', flex_flow='row', justify_content='space-between'),
    style={'description_width': 'initial'},
    tooltip='Haga click para chequear si cumple con las condiciones',
    icon='check'
)

design_guides_button = Button(
    description='Diseño de primers',
    disabled=False,
    button_style='',
    tooltip='Haga click para diseñar los primers',
    icon='dna'
)

conditions_output = Textarea(
    value='',
    placeholder='Los resultados del análisis de condiciones aparecerán aquí',
    description='Output de condiciones:',
    disabled=False,
    layout=Layout(width='auto', height='150px', display='flex', flex_flow='row', justify_content='space-between'),
    style={'description_width': 'initial'}
)

construction_output = Textarea(
    value='',
    placeholder='El diseño de los primers aparecerán aquí',
    description='Output de primers:',
    disabled=False,
    layout=Layout(width='auto', height='250px', display='flex', flex_flow='row', justify_content='space-between'),
    style={'description_width': 'initial'}
)

# Widget en el layout
input_layout = VBox([
    module_dropdown,
    num_guides_input,
    guide_sequence_inputs_box, # Van apareciendo dinamicamente
    enzyme_dropdown,
    HBox([check_conditions_button, design_guides_button])
])

output_layout = VBox([
    conditions_output,
    construction_output
])

full_gui = VBox([input_layout, output_layout])

# Función para que aparezcan dinamicamente
def update_guide_inputs(change):
    num_guides = change['new']
    guide_sequence_inputs_box.children = [
        Text(description=f'Secuencia de guía N° {i+1}:', disabled=False,
             layout=Layout(width='auto', display='flex', flex_flow='row', justify_content='space-between'),
             style={'description_width': 'initial'})
        for i in range(num_guides)
    ]

# Observo cambios en los números de las guías
num_guides_input.observe(update_guide_inputs, names='value')


#Menú principal

#Funciones modificadas para devolver string en lugar de un print

def menu():
  return '--------- Menú principal -----------\n\n1) pMOD_B CSY4\n2) pMOD_B tRNA\n'

def pmod(vector, cant_guias, guias):
  output = ''
  if vector == 'csy4':
    output += '\n--------- pMOD_B CSY4 -----------\n'
  elif vector == 'trna':
    output += '\n--------- pMOD_B tRNA -----------\n'
  return output

def condiciones(guias, vector, ERs, continuar):
  output = '--------- Condiciones -----------\n'
  output += enzrest(guias, ERs) + '\n'
  output += comienzo(guias, vector) + '\n'
  output += nucleotidos(guias) + '\n'


  c1 = enzrest(guias, ERs, return_count=True)
  c2 = comienzo(guias, vector, return_count=True)
  c3 = nucleotidos(guias, return_count=True)


  if c1 + c2 + c3 == 0:
    output += '\nLas secuencias son válidas y cumplen con los requisitos\n\n'
  else:
    output += '>>>>>>>>>>LAS SECUENCIAS NO CUMPLEN CON LOS REQUISITOS!<<<<<<<<<<'

  return output


def ER(ERs):
  output = '\n--------- Enzimas y sitios de restricción -----------\n\n'
  output += '1) Esp3I\n'
  output += '2) BsaI\n\n'
  return output

def enzrest(guias, ERs, return_count=False):
  output = ''
  contador1 = 0
  for guia in guias:
    if ERs == 1:
      ERn=['esp3I', 'sapI', 'aarI']
      ERseq=['CGTCTC', 'GCTCTTC', 'CACCTGC']
      for i in range(0, 2):
        if ERseq[i] in guia:
          output += f'>>> La guía {guia} tiene un sitio de restricción de la enzima {ERn[i]} ({ERseq[i]})\n'
          contador1 += 1

    elif ERs == 2:
      ERn=['bsaI', 'sapI', 'aarI']
      ERseq=['GGTCTC', 'GCTCTTC', 'CACCTGC']
      for i in range(0,2):
        if ERseq[i] in guia:
          output += f'>>> La guía {guia} tiene un sitio de restricción de la enzima {ERn[i]} ({ERseq[i]})\n'
          contador1 += 1
  if contador1 == 0 and not return_count:
    output += '> Las guías no tienen sitios de restricción\n'

  if return_count:
      return contador1
  else:
      return output


def comienzo(guias, vector, return_count=False):
  output = ''
  contador2 = 0
  for guia in guias: #Para cada guía en la lista "guias" corroboro que no empiece con "GGTG"
    if vector == 'csy4':
      if guia.startswith('GTG'):
        output += f'>>> La guía {guia} inicia con GTG\n'
        contador2 += 1

    elif vector == 'trna':
      if guia.startswith('GGTG'):
        output += f'>>> La guía {guia} inicia con GGTG\n'
        contador2 += 1
  if contador2 == 0 and not return_count:
    output += '> Las guías comienzan correctamente\n'

  if return_count:
      return contador2
  else:
      return output


def nucleotidos(guias, return_count=False):
  output = ''
  contador3 = 0
  nucleotidos_9_12 = []
  for guia in guias: #Para cada guía tomo los nucleótidos 9-12 y los agrego a la lista "nucleotidos"
    if len(guia) >= 12:
      nuc912 = guia[8:12]
      nucleotidos_9_12.append(nuc912)
    else:
        output += f">>> La guía {guia} es demasiado corta para verificar nucleótidos 9-12.\n"
        contador3 +=1

  if len(nucleotidos_9_12) != len(set(nucleotidos_9_12)): #Si se repiten los elementos en "nucleotidos" debo imprimir "se repiten"
    output += ">>> Se repiten los nucelótidos de 9-12 en algunas guías\n" #Con la función set() lo que hago es eliminar los duplicados
    contador3 += 1


  if contador3 == 0 and not return_count:
    output += '> Los nucleótidos de 9-12 no se repiten en ninguna guía\n'

  if return_count:
      return contador3
  else:
      return output


#______________________________________________________________________________
#EMPEZAR DESDE ACÁ A HACER EL CÓDIGO DE CONSTRUCCIOÓN

def construccion(guias, vector, ERs):
  output = '\n--------- Diseño de guías -----------\n'
  diseñadas = {}
  revcomp = {'A':'T', 'T':'A', 'G':'C', 'C':'G'}
  for guia in guias:
    guia_revcomp = ''
    for nucleotido in guia:
      nucrevcomp = revcomp[nucleotido]
      guia_revcomp = guia_revcomp + str(nucrevcomp)
    guia_revcomp = guia_revcomp[::-1]
    diseñadas[guia] = guia_revcomp

  FW_ER = ['TCGTCTCA', 'TGGTCTCA']
  RV_ER = ['TCGTCTCC', 'TGGTCTCC']
  REV_VECT = ['CTGCCTATACGGCAGTGAAC', 'TGCACCAGCCGGGAATCG']
  Fw_gral_inicio = FW_ER[ERs-1]
  Fw_gral_ultimo = 'GTTTTAGAGCTAGAAATAGC'
  Rv_gral_inicio = RV_ER[ERs-1]
  if vector == 'csy4':
    Rv_gral_ultimo = REV_VECT[0]
  elif vector == 'trna':
    Rv_gral_ultimo = REV_VECT[1]

  fwfinales = []
  rvfinales = []
  oCmYLCV = 'TGCTCTTCGCGCTGGCAGACATACTGTCCCAC' #Agrego la guía general inicial
  term_seqs = ['TGCTCTTCTGACCTGCCTATACGGCAGTGAAC', 'TGCTCTTCTGACTGCACCAGCCGGGAATCG']
  if vector == 'csy4':
    termfinal = term_seqs[0]
  elif vector == 'trna':
    termfinal = term_seqs[1]

  #Diseño los primers fw

  for guia in guias:
    #Hago guías Fw
    if len(guia) >= 8:
        guiafw = Fw_gral_inicio + guia[8::] + Fw_gral_ultimo
        fwfinales.append(guiafw)
    else:
        output += f">>> La guía {guia} es demasiado corta para diseñar primer Fw.\n"

    #Hago guías Rv
    guiareverse = diseñadas.get(guia, "")
    if len(guiareverse) >= 8:
        guiarv = Rv_gral_inicio + guiareverse[8::] + Rv_gral_ultimo
        rvfinales.append(guiarv)
    else:
         output += f">>> La guía {guia} es demasiado corta para diseñar primer Rv.\n"


  # Imprimir las guías diseñadas y construir la secuencia final
  output += 'Reacción 1:\n'
  output += f'>oCmYLCV\n{oCmYLCV}\n'
  for i in range(0, len(fwfinales)):
    output += f'>Rv{i+1}\n{rvfinales[i]}\n\n'
    output += f'Reacción {i+2}:\n'
    output += f'>Fw{i+1}\n{fwfinales[i]}\n'
  output += f'>term\n{termfinal}\n'

  output += '\n------------ Construcción final ------------\n'

  if vector == 'csy4':
    promCSY = 'CGCGCTGGCAGACATACTGTCCCACAAATGAAGATGGAATCTGTAAAAGAAAACGCGTGAAATAATGCGTCTGACAAAGGTTAGGTCGGCTGCCTTTAATCAATACCAAAGTGGTCCCTACCACGATGGAAAAACTGTGCAGTCGGTTTGGCTTTTTCTGACGAACAAATAAGATTCGTGGCCGACAGGTGGGGGTCCACCATGTGAAGGCATCTTCAGACTCCAATAATGGAGCAATGACGTAAGGGCTTACGAAATAAGTAAGGGTAGTTTGGGAAATGTCCACTCACCCGTCAGTCTATAAATACTTAGCCCCTCCCTCATTGTTAAGGGAGCAAAATCTCAGAGAGATAGTCCTAGAGAGAGAAAGAGAGCAAGTAGCCTAGAAGTAGTCAAGGCGGCGAAGTATTCAGGCACGTGGCCAGGAAGAAGAAAAGCCAAGACGACGAAAACAGGTCCTGCAG'
    CSY = 'GTTCACTGCCGTATAGGCAG'
    scaff = 'GTTTTAGAGCTAGAAATAGCAAGTTAAAATAAGGCTAGTCCGTTATCAACTTGAAAAAGTGGCACCGAGTCGGTGC'
    term_part = 'GTTCACTGCCGTATAGGCAGGTCGATCGACAAGCTCGAGTTTCTCCATAATAATGTGTGAGTAGTTCCCAGATAAGGGAATTAGGGTTCCTATAGGGTTTCGCTCATGTGTTGAGCATATAAGAAACCCTTAGTATGTATTTGTATTTGTAAAATACTHCTATCAATAAAATTTCTAATTCCTAAAACCAAAATCCAGTACTAAAATCCAGATCCCCCGAATTAGAGCTCTACCGGTGGCAGGTCGAAGAGCA' # Renamed variable

    consfinal = promCSY + '/'
    for i in range(0,len(guias)):
      segmento = CSY + '/' + guias[i] + '/' + scaff + '/'
      consfinal = consfinal + segmento

    consfinal = consfinal + term_part
    output += f'\nLa construcción final luego de la reacción GoldenGate será la siguiente: {consfinal}\n'

  elif vector == 'trna':
    output += 'Proximamente...\n'

  return output


def check_conditions_handler(b):
    conditions_output.value = '' # Borro el output anterior
    selected_module = module_dropdown.value
    # Me aseguro que guide_sequence_inputs_box.children es una lista de texto
    if not isinstance(guide_sequence_inputs_box.children, (list, tuple)):
        guide_sequence_inputs_box.children = list(guide_sequence_inputs_box.children)

    guide_sequences = [input_widget.value.strip() for input_widget in guide_sequence_inputs_box.children if isinstance(input_widget, Text)] # Added strip()
    selected_enzyme = enzyme_dropdown.value

    # llamo a la función condiciones
    conditions_result = condiciones(guide_sequences, selected_module, selected_enzyme, '1')

    # Muestro el string resultante en el área de texto
    conditions_output.value = conditions_result


def design_guides_handler(b):
    construction_output.value = '' # Despejo el output anterior
    selected_module = module_dropdown.value
    # Me aseguro que guide_sequence_inputs_box.children es una lista de texto
    if not isinstance(guide_sequence_inputs_box.children, (list, tuple)):
        guide_sequence_inputs_box.children = list(guide_sequence_inputs_box.children)

    guide_sequences = [input_widget.value.strip() for input_widget in guide_sequence_inputs_box.children if isinstance(input_widget, Text)] # Added strip()
    selected_enzyme = enzyme_dropdown.value

    # Llamo a la función construccion
    construction_result = construccion(guide_sequences, selected_module, selected_enzyme)

    # DEvuelvo el string resultante en el área de texto
    construction_output.value = construction_result


# Conecto los botones a las funciones
check_conditions_button.on_click(check_conditions_handler)
design_guides_button.on_click(design_guides_handler)


display(full_gui)
#AGGAGGAGAAAACGCTGCCT
#CAAACTTGAGTCCGAGAGAA
#CAAGAAGAGGAAGCTTAGTG
#GTCCGAAAGAAAGGATAGGC
#Mal
#GTCCGAAAGAAAGGATAGGC
#ACTAAGCTGAAACTTCTTGT
#CGTAACAAGAAGAACAAAGG

#Instrucciones

1) La opción módulo se refiere a la región de corte que se utilizará, teniendo las opciones CSY4 o tRNA.

2) En la sección de número de guías se debe colocar la cantidad de guías diseñadas que estarán en el mismo vector. Se puede aumentar o disminuir con las flechas de la derecha o escribiendo la cantidad con el teclado.

3) Al colocar el número de guías diseñadas, aparecerán las celdas en las que se solicitará la secuencia de las guías diseñadas.
**IMPORTANTE:** La secuencia de la guía no debe contener NGG!

4) La opción de enzima de restricción se refiere a la enzima de restricción que se utilizará para realizar el clonado en el pModB. Puede ser Esp3I o BsaI.

5) Luego de haber ingresado todas las variables, se puede realizar un análisis de condiciones de las guías diseñadas, donde se analiza:
- (i) que no comiencen con 'GTG' en caso de que se use módulo CSY4 o 'GGTG' en caso de que se utilice el módulo tRNA para evitar generación de sitios de restricción;
- (ii) que los nucleótidos 9 al 12 de las guías diseñadas no se repitan para evitar posibles hibridaciones;
- (iii) que las guías no posean sitio de reconocimiento de la enzima de restricción elegida para evitar cortes indeseados.

**IMPORTANTE**: En caso de que no cumplan con alguno de los requisitos se dará aviso al usuario mediante el siguiente mensaje:

">>>>>>>>>>LAS SECUENCIAS NO CUMPLEN CON LOS REQUISITOS!<<<<<<<<<<"

Además, se dará aviso de cuál es la condición que no se cumpla. Si esto sucede, se recomienda el cambio de las guías para evitar futuros errores en la construcción del vector de edición.

6) Luego de haber confirmado que las guías diseñadas cumplen con las condiciones se puede proseguir a realizar el diseño de los primers. Para esto se debe clickear en el botón de diseño de primers y se otorgarán los primers diseñados a partir de las guías para realizar el Golden Gate. Los resultados otorgados se verán en forma de reacciones, donde el programa te organiza las reacciones de PCR que se deben realizar para tener cada uno de los fragmentos para la GoldenGate. Los primers que contienen los nombres >oCmYLCV y >term (primero y último) son los primers generales que no varían. Luego, los primers Fw1 y Rv1 son aquellos diseñados a partir de la guía 1, los Fw2 y Rv2 son los diseñados a partir de la guía 2 y así sucesivamente.

**IMPORTANTE**: En el caso de que se use el módulo CSY4, el programa otorgará, además, la secuencia del inserto luego de la Golden Gate inicial, en donde se podrán diferenciar los siguientes elementos: Promotor, guías (dependiendo de cuantas se hayan diseñado), CSY4 y scaffold separados cada uno por un '/' para poder diferenciarlos. Esta funcionalidad aun no está disponible para el módulo tRNA.